In [498]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import numpy as np

%matplotlib inline  

In [499]:
account = pd.read_csv('raw/account.csv',sep=';')
card = pd.read_csv('raw/card_dev.csv',sep=';')
client = pd.read_csv('raw/client.csv',sep=';')
disp = pd.read_csv('raw/disp.csv', sep=';')
district = pd.read_csv('raw/district.csv', sep=';')
loan = pd.read_csv('raw/loan_dev.csv', sep=';')
trans = pd.read_csv('raw/trans_dev.csv', sep=';')

c:\Users\ASUS\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Data processing

### Extract gender and birth date from birth number

The gender and birth date can be extracted from the birth number.

In [500]:
# YYMMDD for men
# YYMM+50DD for women

client['gender'] = client.apply(lambda row: 'M' if (row['birth_number']//100)%100 < 50 else 'F', axis=1)
client['birth_date'] = client.apply(lambda row: "19{:02d}-{:02d}-{:02d}".format((row['birth_number']//10000)%100,((row['birth_number']//100)%100 if row['gender'] == 'M' else (row['birth_number']//100)%100 - 50), (row['birth_number'])%100), axis=1)
client.drop('birth_number', inplace=True, axis=1)
client['birth_date'] = client['birth_date'].apply(pd.to_datetime)

client

,client_id,district_id,gender,birth_date
0,1,18,F,1970-12-13
1,2,1,M,1945-02-04
2,3,1,F,1940-10-09
3,4,5,M,1956-12-01
4,5,5,F,1960-07-03
...,...,...,...,...
5364,13955,1,F,1945-10-30
5365,13956,1,M,1943-04-06
5366,13968,61,M,1968-04-13
5367,13971,67,F,1962-10-19


In [501]:
def formatDate(df, column):
    df[column] = df[column].apply(lambda cell: "19{:02d}-{:02d}-{:02d}".format((cell//10000)%100, (cell//100)%100, cell%100))
    df[column] = df[column].apply(pd.to_datetime)

formatDate(account, "date")
formatDate(loan, "date")
formatDate(trans, "date")
formatDate(card, "issued")

### Separate a date's fields

We can divide the date column into day, year, month and day of week columns.

In [ ]:
def processDate(df, column):
    df[column + "_year"]     = df[column].dt.year
    df[column + "_month"]    = df[column].dt.month
    df[column + "_day"]      = df[column].dt.day
    df[column + "_day_week"] = df[column].dt.dayofweek
    #df.drop(column, inplace=True, axis=1)

processDate(account, "date")
processDate(client, "birth_date")
processDate(loan, "date")
processDate(trans, "date")
processDate(card, "issued")

### Nulls

#### Account

In [ ]:
account.isnull().sum()

#### Client

In [ ]:
client.isnull().sum()

#### Loan

In [ ]:
loan.isnull().sum()

#### Transaction

In [ ]:
trans.isnull().sum()

#### Card

In [ ]:
card.isnull().sum()

#### Disposition

In [ ]:
disp.isnull().sum()

#### District

In [ ]:
district.isnull().sum()

### Merging tables

#### Merging client table with district table

In [ ]:
client_district = client.merge(district, left_on='district_id', right_on='code ')
client_district

#### Merging Account table with Client table

In [ ]:
account_client = disp.merge(client, left_on="client_id", right_on='client_id')
account_client = account_client.merge(account, left_on="account_id", right_on="account_id")
account_client.rename({"date": "account_creation_date", 
                        'date_year': 'account_creation_year', 
                        'date_day': 'account_creation_day', 
                        "date_month": "account_creation_month", 
                        "date_day_week": "account_creation_day_week"}, axis=1, inplace=True)
display(account_client)

#### Merging Loan table with Account table

In [ ]:
loan_account = account_client.merge(loan, left_on="account_id", right_on='account_id', how="inner")
loan_account.rename({"date": "loan_date", 
                        'date_year': 'loan_year', 
                        'date_day': 'loan_day', 
                        "date_month": "loan_month", 
                        "date_day_week": "loan_day_week"}, axis=1, inplace=True)

# remove the disponent entries
loan_account_owners = loan_account[loan_account["type"]!="DISPONENT"]
loan_account_owners
display(loan_account.columns)
loan_account.frequency

#### Merging card table with client table

In [ ]:
disp2 = disp.rename(columns={"type": "disp_type"})
card2 = card.rename(columns={"type": "card_type"})

card_disp = card2.merge(disp2, how="outer")
card_disp_client = pd.merge(card_disp, client).drop_duplicates(subset=['account_id'])
card_disp_client

### Adding client age column to loan

In [ ]:
import datetime
from datetime import datetime, date
def age(born_date_str, loan_date_str):
    born_date = datetime.strptime(str(born_date_str).split(" ")[0], "%Y-%m-%d").date()
    loan_date = datetime.strptime(str(loan_date_str).split(" ")[0], "%Y-%m-%d").date()

    return loan_date.year - born_date.year - ((loan_date.month, 
                                      loan_date.day) < (born_date.month, 
                                                    born_date.day))
  
col = loan_account_owners.apply(lambda row: age(row["birth_date"], row["loan_date"]), axis=1)
loan_account_owners = loan_account_owners.assign(client_age=col.values)
loan_account_owners["client_age"]


#### Merging client table

In [ ]:
#disp_client_dist = disp.merge(client_district, left_on="client_id", right_on="client_id")
#complete = pd.merge(account, pd.DataFrame(trans.groupby('account_id').size(), columns=['# trans']), right_index=True, left_on='account_id', how="outer")
#display(complete)


# Merging Accounts' and Transactions' Data

import numpy


TRANS_OP_1 = "credit in cash"
TRANS_OP_2 = "credit card withdrawal"
TRANS_OP_3 = "collection from another bank"
TRANS_OP_4 = "remittance to another bank"
TRANS_OP_5 = "withdrawal in cash"

TRANS_TYPE_1 = "withdrawal"
TRANS_TYPE_2 = "withdrawal in cash"
TRANS_TYPE_3 = "credit"

complete = pd.merge(account, pd.DataFrame(trans.groupby('account_id').size(), columns=['trans_num']), right_index=True, left_on='account_id', how="outer")
complete = pd.merge(complete, pd.DataFrame(trans[trans['operation']=='credit in cash'].groupby('account_id').size(), columns=['trans_op_1_num']), right_index=True, left_on='account_id', how="outer")
complete = pd.merge(complete, pd.DataFrame(trans[trans['operation']=='credit card withdrawal'].groupby('account_id').size(), columns=['trans_op_2_num']), right_index=True, left_on='account_id', how="outer")
complete = pd.merge(complete, pd.DataFrame(trans[trans['operation']=='collection from another bank'].groupby('account_id').size(), columns=['trans_op_3_num']), right_index=True, left_on='account_id', how="outer")
complete = pd.merge(complete, pd.DataFrame(trans[trans['operation']=='remittance to another bank'].groupby('account_id').size(), columns=['trans_op_4_num']), right_index=True, left_on='account_id', how="outer")
complete = pd.merge(complete, pd.DataFrame(trans[trans['operation']=='withdrawal in cash'].groupby('account_id').size(), columns=['trans_op_5_num']), right_index=True, left_on='account_id', how="outer")

complete = pd.merge(complete, pd.DataFrame(trans[trans['type']=='withdrawal'].groupby('account_id').size(), columns=['trans_type_1_num']), right_index=True, left_on='account_id', how="outer")
complete = pd.merge(complete, pd.DataFrame(trans[trans['type']=='withdrawal in cash'].groupby('account_id').size(), columns=['trans_type_2_num']), right_index=True, left_on='account_id', how="outer")
complete = pd.merge(complete, pd.DataFrame(trans[trans['type']=='credit'].groupby('account_id').size(), columns=['trans_type_3_num']), right_index=True, left_on='account_id', how="outer")

complete = pd.merge(complete, pd.DataFrame(trans.groupby('account_id').amount.mean().round(2)).rename(columns={'amount':'avg trans amount'}), right_index=True, left_on='account_id', how="outer")
complete = pd.merge(complete, pd.DataFrame(trans.groupby('account_id').balance.mean().round(2)).rename(columns={'balance':'avg trans balance'}), right_index=True, left_on='account_id', how="outer")
# (4500, 15)
display(complete)
'''
# ------------------------


# Merging with Loans' Data
complete = pd.merge(complete, loan.rename(columns={'date': 'loan date', 'amount': 'loan amount', 'duration': 'loan duration (mths)', 'payments': 'loan payments', 'status': 'loan status'}))
# (328, 21)

# ------------------------

# Merging with Districts' Data
complete = pd.merge(complete, district.rename(columns={'code ': 'district_id', 'name ': 'district_name', 'region': 'district_region'}))
# (328, 36)

# ------------------------

# Merging Disp' and Cards' Data
disp = disp.rename(columns={"type": "disp type"})
disp['has disponent'] = numpy.where(disp['disp type'] == 'DISPONENT', True, False)
card = card.rename(columns={"type": "card type"})
disp_card = pd.merge(disp, card, how="outer")
# (5369, 9)

# ------------------------

# Merging Previous with Clients' Data
disp_card_clie = pd.merge(disp_card, client).drop_duplicates(subset=['account_id']).drop(columns=['has disponent', 'disp type', 'disp_id'])
# (4500, 7)
temp = pd.merge(disp_card_clie, pd.DataFrame(disp_card.groupby('account_id').size(), columns = ['count owner']), right_index=True, left_on='account_id', how="outer")
disp_card_clie['has disponent'] = numpy.where(temp['count owner'] == 2, True, False)
# (4500, 8)

# ------------------------

# Merging Previous subset with the rest
complete = pd.merge(complete, disp_card_clie)


print(complete.shape)
complete

'''


#### Merging 

### Add No card type to Card client table

In [ ]:
card_disp_client.loc[card_disp_client["card_type"].isna()] = "no card"
card_disp_client["card_type"]

### Add customer ratio column to district table

In [ ]:
district_client = pd.DataFrame(client_district.groupby(['name ', "no. of inhabitants"], axis = 0)['name '].count())
district_client.columns = ['count']
district_client.reset_index(inplace=True)
district_client['client_ratio'] = district_client['count'] / district_client["no. of inhabitants"] * 100
district = district.merge(district_client[["name ","client_ratio"]], left_on='name ', right_on='name ')
district

### Add inhabitant number (in millions) column to district table

In [ ]:
district["no. of inhabitants_mln"] = district["no. of inhabitants"] / 1000000
district

# Data analysis

### Transaction statistics

In [ ]:
trans.describe()

### District statistics

In [ ]:
district.describe()

### District inhabitant, average salary, customer ratio

In [ ]:
#plotting Number of inhabitants vs. Average salary vs. Customers proportion within population of districts

sns.set_style(style="darkgrid")

## Initialize the matplotlib figure:
fig2, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(12,16), sharey=False)
fig2.tight_layout()  #When working with 'tight_layout', the subplot must be adjusted [https://stackoverflow.com/questions/7066121/how-to-set-a-single-main-title-above-all-the-subplots-with-pyplot]
fig2.subplots_adjust(top=0.96)  #Adjusting the space for the superior title

## Plot the District population
sns.set_color_codes("muted")
sns.barplot(x="no. of inhabitants_mln", y="name ", data=district, label = 'District population (mln)',color="skyblue", ax=ax1)
## Add a legend and informative axis label
ax1.legend(ncol=2, loc="lower right", frameon=True)
ax1.set(xlim=(0, 1.21), ylabel="", xlabel="District population (mln)")
sns.despine(left=True, bottom=True,ax=ax1)

## Plot the Average salary
sns.set_color_codes("muted")
sns.barplot(x="average salary ", y="name ", data=district, label="Average salary", color="skyblue", ax=ax2)
## Add a legend and informative axis label
ax2.legend(ncol=2, loc="lower right", frameon=True)
ax2.set(xlim=(0, 12541), ylabel="", xlabel="average salary ")  #The xlim value comes from the maximum value in the dataset.
ax2.set_yticklabels([''])
sns.despine(left=True, bottom=True, ax=ax2)

sns.set_color_codes("muted")
sns.barplot(x="client_ratio", y="name ", data=district, label="% of clients in population", color="skyblue", ax=ax3)
## Add a legend and informative axis label
ax3.legend(ncol=2, loc="lower right", frameon=True)
ax3.set(xlim=(0, 0.145), ylabel="", xlabel="Relative number of clients comparing to district population (%)")  #The xlim value comes from the maximum value in the dataset.
ax3.set_yticklabels([''])
sns.despine(left=True, bottom=True, ax=ax3)

plt.suptitle('Number of inhabitants vs. Average salary vs. Customer ratio', fontsize=14, fontweight='bold')
plt.show()

### Loan amount vs payment duration

In [ ]:
#sns.set_style(style="darkgrid")
#fig = plt.gcf()
#fig.set_size_inches(7, 7)
#display(loan)
#sns.scatterplot(data=loan, x="duration", y="amount", color="red")
#plt.show()




### Gender proportion

In [ ]:
colors = sns.color_palette('bright')[0:5]
plt.pie(client['gender'].value_counts(), labels=['M', 'F'], colors=colors, autopct='%1.1f%%')
plt.show()

sns.set_style(style="darkgrid")
fig = plt.gcf()
fig.set_size_inches(7, 7)
sns.countplot(data=client, x="gender", color="skyblue", order=client["gender"].value_counts().index )
plt.show()

### Age at loan distribution

In [ ]:
sns.set_style(style="darkgrid")
fig = plt.gcf()
fig.set_size_inches(7, 7)

hp = sns.histplot(data=loan_account_owners, x="client_age", color="skyblue", binwidth=2, stat="density")
kp = sns.kdeplot(data=loan_account_owners, x="client_age", color="blue", ax=hp, label="Kernel density estimation")
#plt.title(fig_title)
plt.legend()

### Loan status proportion

In [ ]:
colors = sns.color_palette('bright')[0:5]
plt.pie(loan['status'].value_counts(), labels=['payed off', 'in debt'], colors = colors, autopct='%1.1f%%')
plt.show()

sns.set_style(style="darkgrid")
fig = plt.gcf()
fig.set_size_inches(7, 7)
# TODO
sns.countplot(data=loan, x="status", color="skyblue", order=loan["status"].value_counts().index )
plt.show()


### Card type proportion

In [ ]:
colors = sns.color_palette('bright')[0:5]
plt.pie(card['type'].value_counts(), labels=["classic", "gold", "junior"], colors = colors, autopct='%1.1f%%')
plt.show()

sns.set_style(style="darkgrid")
fig = plt.gcf()
fig.set_size_inches(7, 7)
sns.countplot(data=card, x="type", color="skyblue", order=card["type"].value_counts().index )
plt.show()


colors = sns.color_palette('bright')[0:5]
plt.pie(card_disp_client['card_type'].value_counts(), labels=["classic", "gold", "junior", "no card"],colors = colors, autopct='%1.1f%%')
plt.show()

sns.set_style(style="darkgrid")
fig = plt.gcf()
fig.set_size_inches(7, 7)
sns.countplot(data=card_disp_client, x="card_type", color="skyblue", order=card_disp_client["card_type"].value_counts().index )
plt.show()
#plt.title()
  

### Card type by sex

In [ ]:
sns.set_style(style="darkgrid")

card_disp_client.loc[card_disp_client["card_type"] == "no card"] = np.nan


# Draw a nested barplot by species and sex
g = sns.catplot(
    data=card_disp_client, kind="count",
    x="card_type", hue="gender",
    errorbar="sd", palette="dark", alpha=.6, height=6
)
g.despine(left=True)
g.set_axis_labels("", "Count")
g.legend.set_title("")

### Account issuance

In [ ]:
sns.set_style(style="darkgrid")
fig = plt.gcf()
fig.set_size_inches(7, 7)
sns.countplot(data=account, x="frequency", color="skyblue", order=account["frequency"].value_counts().index )
plt.show()

### Urban inhabitants vs inhabitant number and average salary

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(district, x="no. of inhabitants", y="ratio of urban inhabitants ", hue="average salary ", palette="magma" )
norm = plt.Normalize(district["average salary "].min(), district["average salary "].max())
sm = plt.cm.ScalarMappable(cmap="magma", norm=norm)
sm.set_array([])

# Remove the legend and add a colorbar
ax.get_legend().remove()
ax.figure.colorbar(sm)
#ax.set_xlabel("no. of inhabitants")
plt.show()

### Loan amount distribution

In [ ]:
sns.set_style(style="darkgrid")
fig = plt.gcf()
fig.set_size_inches(7, 7)

hp = sns.histplot(data=loan, x="amount", color="skyblue", binwidth=10000, stat="density")
kp = sns.kdeplot(data=loan, x="amount", color="blue", ax=hp, label="Kernel density estimation")
#plt.title(fig_title)
plt.legend()
plt.show()

### Average salary distribution

In [ ]:
sns.set_style(style="darkgrid")
fig = plt.gcf()
fig.set_size_inches(7, 7)

hp = sns.histplot(data=district, x="average salary ", color="skyblue", binwidth=500, stat="density")
kp = sns.kdeplot(data=district, x="average salary ", color="blue", ax=hp, label="Kernel density estimation")
#plt.title(fig_title)
plt.legend()
plt.show()

### 

In [ ]:
fig = plt.figure()
fig.set_size_inches(9, 9)

ax = fig.add_subplot(111, projection = '3d')

x = district["ratio of urban inhabitants "]
y = district["unemploymant rate '96 "]
z = district["no. of inhabitants"]
c = district["average salary "]

ax.set_xlabel("ratio of urban inhabitants")
ax.set_ylabel("unemployment rate")
ax.set_zlabel("number of inhabitants")

img = ax.scatter(x, y, z, c=c, cmap="magma")
fig.colorbar(img)
plt.show()

### 

In [ ]:
trans_acc = pd.merge(trans, account, on='account_id')
trans_acc.head()
#transactions per account
max(trans_acc['account_id'].value_counts())

In [ ]:
loan_acc = pd.merge(loan, account, on='account_id')
print(loan.shape[0], account.shape[0])
max(loan_acc['account_id'].value_counts())

In [ ]:
no_owners_per_account = disp['account_id'].value_counts()
print(no_owners_per_account.max())
print(no_owners_per_account.min())
print(no_owners_per_account.std())
print(no_owners_per_account.mean())